In [ ]:
import tensorflow as tf
import numpy as np

TRG_TRAIN_DATA = "index20wtrain.resp"  # 答案输入语言
ENC_TRG_DATASET = "enc_trg_dataset.txt"
CHECKPOINT_PATH = "./IR_ckpt-5200"
TRG_TRAIN_DATA_SIZE = 200000


HIDDEN_SIZE = 300           # LSTM的隐藏层规模
NUM_LAYERS = 2              # 深层循环神经网络中的LSTM结构的层数
VOCAB_SIZE = 10000          # 词汇表的大小（词汇表按照词频，由高到低向下排列）
MAX_LEN = 50                # 限定句子的最大单词数量。
BATCH_SIZE = 1              # 训练数据batch的大小

def MakeDataset(file_path):
    dataset = tf.data.TextLineDataset(file_path)
    # 根据空格将单词编号切分开并放入一个一维向量。
    dataset = dataset.map(lambda string: tf.string_split([string]).values)
    # 将字符串形式的单词编号转化为整数。
    dataset = dataset.map(
        lambda string: tf.string_to_number(string, tf.int32))
    # 统计每个句子的单词数量，并与句子内容一起放入Dataset中。
    dataset = dataset.map(lambda x: (x, tf.size(x)))

    return dataset


def MakeTrgDataset(trg_path, batch_size):
    # 统计src_path所含的总行数（src_path与trg_path所含行数相等）
    trg_data = MakeDataset(trg_path)
    # 规定填充后输出的数据维度。
    padded_shapes = (
        (tf.TensorShape([None]),  # 源句子是长度未知的向量
         tf.TensorShape([])))  # 标签是单个数字
    # 调用padded_batch方法进行batching操作。
    batched_dataset = trg_data.padded_batch(batch_size, padded_shapes)
    return batched_dataset

# 定义IRModel类来描述模型
class IRModel(object):
    # 在模型的初始化函数中定义模型要用到的变量
    def __init__(self):
        # 定义编码器的LSTM结构
        self.enc_cell = tf.nn.rnn_cell.MultiRNNCell(
        [tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE)
         for _ in range(NUM_LAYERS)])

        # 为源语言和目标语言分别定义词向量
        self.embedding = tf.get_variable("src_emb", [VOCAB_SIZE, HIDDEN_SIZE])
        self.W = tf.get_variable("weights", [HIDDEN_SIZE, HIDDEN_SIZE],
                                 initializer=tf.truncated_normal_initializer())

    # trg_input,trg_size,targets分别是上面MakeSrcTrgDataset函数产生的五种张量。
    def create_enc_trg(self, trg_input, trg_size):

        trg_emb = tf.nn.embedding_lookup(self.embedding, trg_input)

        # 使用dynamic_rnn构造编码器。
        with tf.variable_scope("encoder"):
            enc_trg_outputs, enc_trg_state = tf.nn.dynamic_rnn(self.enc_cell, trg_emb, sequence_length=trg_size,
                                                               dtype=tf.float32)
            # 因为编码器是一个双层LSTM，因此enc_state是一个包含两个LSTMStateTuple类的tuple，enc_state存储的是每一层的最后一个step的输出
            # trg_enc_output的shape为BATCH_SIZE * HIDDEN_SIZE
            trg_enc_output = enc_trg_outputs[:, -1, :]

        return trg_enc_output


def main():
    with tf.variable_scope("IRModel", reuse=None):
        model = IRModel()
    # 定义输入数据。
    data = MakeTrgDataset(TRG_TRAIN_DATA,  BATCH_SIZE)
    iterator = data.make_initializable_iterator()

    (trg_input, trg_size) = iterator.get_next()
    # 定义前向计算图。输入数据以张量形式提供给forward函数。
    trg_output = model.create_enc_trg(trg_input, trg_size)

    sess = tf.Session()
    saver = tf.train.Saver()
    saver.restore(sess, CHECKPOINT_PATH)
    sess.run(iterator.initializer)
    def matrixAppend(sess, row, col):
        outputs = np.zeros(shape=(row, col))
        for num in range(row):
            output = sess.run(trg_output)
            outputs[num] = output[0]
        return outputs

    dataset = matrixAppend(sess, TRG_TRAIN_DATA_SIZE, HIDDEN_SIZE)
    np.savetxt(ENC_TRG_DATASET, dataset)
    sess.close()


if __name__ == "__main__":
    main()


